## Setup

In [ ]:
%pip install 'vanna[qdrant,openai,mysql]'

In [3]:
from vanna.openai import OpenAI_Chat
from vanna.qdrant import Qdrant_VectorStore
from qdrant_client import QdrantClient
from openai import OpenAI


In [8]:
class MyVanna(Qdrant_VectorStore, OpenAI_Chat):
    def __init__(self, client=None, config=None):
        # 分别初始化两个父类
        Qdrant_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self,client=client,config=config)

# 配置信息
api_key = "sk-pH3LeinCFI54OlJI238c014eE8D64a0f882cA285375d72A3"
base_url = "https://api.juheai.top/v1"
model_name = "gpt-4.1-nano"

# 初始化Qdrant客户端
config = {
	'client': QdrantClient(host="112.124.5.73",port=2280),
  'model': model_name
}

# 创建Vanna实例
vn = MyVanna(
    client=OpenAI(api_key=api_key,base_url=base_url),
    config=config
)


In [9]:
vn.connect_to_mysql(host='rm-bp1x9482wcuc92dfe6o.mysql.rds.aliyuncs.com', dbname='cocoteam-cloud', user='cocoteam_read', password='rQqSJlZRi9', port=3306)

## Training
You only need to train once. Do not train again unless you want to add more training data.

In [ ]:

# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
# vn.train(plan=plan)



In [ ]:

# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
# vn.train(ddl="""
#     CREATE TABLE `departments` (
#     `department_id` int NOT NULL AUTO_INCREMENT COMMENT '部门ID，主键',
#     `department_name` varchar(50) COLLATE utf8mb4_general_ci NOT NULL COMMENT '部门名称',
#     `location` varchar(100) COLLATE utf8mb4_general_ci DEFAULT NULL COMMENT '部门办公地点',
#     `description` text COLLATE utf8mb4_general_ci COMMENT '部门描述',
#     `created_at` timestamp NULL DEFAULT CURRENT_TIMESTAMP COMMENT '创建时间',
#     PRIMARY KEY (`department_id`)
#   ) ENGINE=InnoDB AUTO_INCREMENT=11 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci COMMENT='公司部门信息表';

# """)

# Sometimes you may want to add documentation about your business terminology or definitions.
# vn.train(documentation="Our business defines OTIF score as the percentage of orders that are delivered on time and in full")

# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(question="每个部门有多少员工？",sql="SELECT d.department_name, COUNT(u.user_id) AS employee_count FROM departments d LEFT JOIN users u ON d.department_id = u.department_id GROUP BY d.department_id;")
vn.train(question="最畅销的商品是什么？",sql="SELECT p.product_name, SUM(o.quantity) AS total_sold FROM products p JOIN orders o ON p.product_id = o.product_id GROUP BY p.product_id ORDER BY total_sold DESC LIMIT 1;")
vn.train(question="按类别统计所有商品的库存总量。",sql="SELECT category, SUM(stock_quantity) AS total_stock FROM products GROUP BY category;")
vn.train(question="显示目前上架的所有商品及其价格。",sql="SELECT product_name, price FROM products WHERE is_active = 1;")
vn.train(question="过去一个月内销售额最高的商品是哪一个？",sql="SELECT p.product_name, SUM(p.price * o.quantity) AS total_sales FROM products p JOIN orders o ON p.product_id = o.product_id WHERE o.order_date BETWEEN DATE_SUB(CURDATE(), INTERVAL 1 MONTH) AND CURDATE() GROUP BY p.product_id ORDER BY total_sales DESC LIMIT 1;")
vn.train(question="哪些用户在过去的一年内没有进行任何购买？",sql="SELECT u.username FROM users u LEFT JOIN orders o ON u.user_id = o.user_id AND o.order_date >= DATE_SUB(CURDATE(), INTERVAL 1 YEAR) WHERE o.order_id IS NULL;")



'be7dea21-0343-5f08-b1dd-c81189b62d1d-sql'

In [7]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type
0,08662319-184c-510d-9477-114ec71fb61d-sql,最近一周新增的线索数量,SELECT COUNT(*) AS recent_week_leads FROM coco...,sql
1,0890ef88-61ab-5366-a3c4-911a1bbca976-sql,按线索来源统计数量,SELECT CASE salesleads_source WHEN 0 THEN '客户推...,sql
2,0d413456-68e6-5cf9-98ee-8b19d538dc07-sql,哪些线索的下次跟进时间在今天之内？,"SELECT s.f_id, s.salesleads_name, u.f_real_nam...",sql
3,18d0bab7-352a-5717-818f-ea13bea6f1c8-sql,哪些线索没有转换为商机？,"SELECT s.f_id, s.salesleads_name, u.f_real_nam...",sql
4,52fb7522-5965-59d8-877e-32271b499182-sql,跟进中的线索有哪些，按负责人统计,"SELECT u.f_real_name AS owner_name, COUNT(*) A...",sql
5,93cb4379-84fd-509b-bc6a-4f8c9003383c-sql,按线索来源统计数量,"SELECT salesleads_source, COUNT(*) AS count FR...",sql
6,9ce027fb-8417-5aef-b819-fefb736c5124-sql,跟进记录最多的前5条线索,"SELECT r.f_salesleads_id, s.salesleads_name, C...",sql
7,ab853aad-8df6-54c9-b7fc-088c90f1c1fa-sql,每个负责人的线索数量是多少？,"SELECT u.f_real_name AS owner_name, COUNT(*) A...",sql
8,b10e03ce-4904-50e2-9b4c-b1e6e6c9f802-sql,哪些线索已经超过了下一个跟进时间但还未跟进？,"SELECT s.f_id, s.salesleads_name, u.f_real_nam...",sql
9,b40eacb1-6d76-5fe9-9737-30fbe4085df9-sql,哪些线索已经被取消？并列出取消原因,"SELECT s.f_id, s.salesleads_name, u.f_real_nam...",sql


In [ ]:
# You can remove training data if there's obsolete/incorrect information.
vn.remove_training_data(id='1-ddl')


## Asking the AI
Whenever you ask a new question, it will find the 10 most relevant pieces of training data and use it as part of the LLM prompt to generate the SQL.

In [10]:
vn.ask(question="按类别统计所有商品的库存总量。")

SQL Prompt: [{'role': 'system', 'content': 'You are a SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE `base_user` (\n  `f_id` varchar(50) NOT NULL COMMENT \'自然主键\',\n  `f_account` varchar(50) DEFAULT NULL COMMENT \'账户\',\n  `f_real_name` varchar(50) DEFAULT NULL COMMENT \'姓名\',\n  `f_quick_query` varchar(100) DEFAULT NULL COMMENT \'快速查询\',\n  `f_nick_name` varchar(50) DEFAULT NULL COMMENT \'呢称\',\n  `f_head_icon` longtext COMMENT \'头像\',\n  `f_gender` int DEFAULT NULL COMMENT \'性别\',\n  `f_birthday` datetime DEFAULT NULL COMMENT \'生日\',\n  `f_mobile_phone` varchar(20) DEFAULT NULL COMMENT \'手机\',\n  `f_tele_phone` varchar(20) DEFAULT NULL COMMENT \'电话\',\n  `f_landline` varchar(50) DEFAULT NULL COMMENT \'F_Landline\',\n  `f_email` varchar(50) DEFAULT NULL COMMENT \'邮箱\',\n  `f_nation` varchar(50) DEFAULT NULL COMMENT \'民族

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn,allow_llm_to_see_data=True,logo="https://cocoteam.com/resource/img/logo@2x.png",title="芯港AI报表测试",subtitle="text2sql在线测试")
app.run()

Google Colab doesn't support running websocket servers. Disabling debug mode.
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

Your app is running at:
https://8084-m-s-3ar3m3lw26l2d-c.us-west1-0.prod.colab.dev
 * Serving Flask app 'vanna.flask'
 * Debug mode: off
